In [2]:
import tensorflow as tf
from json import loads
import pandas as pd
import numpy as np
from glob import glob

In [5]:
df = pd.DataFrame()
datafiles = glob("LOBdata*.csv")

for i in datafiles:
    tdf = pd.read_csv(i)
    df = pd.concat([df,tdf],ignore_index=True)

coins = list(df['coin'].unique())
coins.sort()


x = []
y = []
for i in range(len(df)):
    temp = np.array(loads(df.iloc[i].matrix))
    t = [0]*(len(coins))
    if df.iloc[i].label == True:
        t[coins.index(df.iloc[i].coin)] = 1
        y.append(t)
    else:
        # t[coins.index(df.iloc[i].coin)] = 0
        y.append(t)
X = tf.reshape(tf.convert_to_tensor(x),[len(df),1, 2000])
Y = tf.reshape(tf.convert_to_tensor(y),[len(df),len(coins)])

In [5]:
Y.shape

TensorShape([180, 90])

In [6]:
try:
    del model
except:
    pass
K = 1000
D = 1
s = (1,D,2*K)
# x = tf.keras.layers.Input(shape=(1,2*D,2*K))
# d = tf.random.normal(s)
# d = tf.reshape(d,[1]+list(s))
d = tf.keras.layers.Input(list(s))
# x = tf.keras.layers.Conv2D(8,(11,int((2*K)/2)),activation='relu')(d)
# x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dense(32,activation='relu')(x)
x = tf.keras.layers.Dense(len(coins),activation='softmax')(d)
x = tf.keras.layers.Dense(len(coins),activation='softmax')(x)
x = tf.keras.layers.Dense(len(coins),activation='softmax')(x)
x = tf.keras.layers.Dense(len(coins),activation='softmax')(x)
x = tf.keras.layers.Dense(len(coins),activation='softmax')(x)
x = tf.keras.layers.Dense(len(coins),activation='softmax')(x)
model = tf.keras.Model(inputs=d,outputs=x)





In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 60, 2000, 1)]  0         
                                                                 
 conv2d (Conv2D)             (None, 1, 50, 1001, 8)    88008     
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 40, 335, 8)     469576    
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 30, 113, 8)     157000    
                                                                 
 conv2d_3 (Conv2D)           (None, 1, 20, 39, 4)      26404     
                                                                 
 conv2d_4 (Conv2D)           (None, 1, 10, 14, 2)      2290      
                                                                 
 conv2d_5 (Conv2D)           (None, 1, 1, 6, 1)        181   

In [9]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
   1e-1,20, 0.99, name=None
)
adam = tf.keras.optimizers.Adam(
    learning_rate=lr_schedule,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-4,
    amsgrad=False,
    name='Adam'
)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['acc',tf.keras.metrics.Precision(thresholds=None, top_k=None, class_id=None, name=None, dtype=None),tf.keras.metrics.Recall(
    thresholds=None, top_k=None, class_id=None, name=None, dtype=None
)])
model.fit(x=X,y=Y,batch_size=16,epochs=200,shuffle=True)

Epoch 1/200
12/12 [==============================] - 41s 3s/step - loss: 4.5055 - acc: 0.0000e+00 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00
Epoch 2/200
12/12 [==============================] - 40s 3s/step - loss: 4.2974 - acc: 0.0167 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00
Epoch 3/200
12/12 [==============================] - 40s 3s/step - loss: 4.2369 - acc: 0.0111 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00
Epoch 4/200
12/12 [==============================] - 40s 3s/step - loss: 4.2249 - acc: 0.0222 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00
Epoch 5/200
12/12 [==============================] - 40s 3s/step - loss: 4.2161 - acc: 0.0222 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00
Epoch 6/200
12/12 [==============================] - 40s 3s/step - loss: 4.2155 - acc: 0.0056 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00
Epoch 7/200
12/12 [==============================] - 40s 3s/step - loss: 4.2117 - acc: 0.0389 - precision_1: 0.0000e+00 - recall_1: 0.0000

In [43]:
df = pd.read_csv("LOBdata_test_.csv")
x = []
y = []
for i in range(len(df)):
    x.append(tf.convert_to_tensor(np.reshape(np.array(loads(df.iloc[i].matrix)),(120,2000,1))))
    if df.iloc[i].label == True:
        y.append([1,0])
    else:
        y.append([0,1])
X_test = tf.reshape(tf.convert_to_tensor(x),[len(df),1 , 120, 2000, 1])
Y_test = tf.reshape(tf.convert_to_tensor(y),[len(df),2])


[]

In [39]:
tf.argmax( model.predict(X_test),axis=1)

<tf.Tensor: shape=(45,), dtype=int64, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1], dtype=int64)>

In [38]:
tf.linalg.normalize(X)

InvalidArgumentError: Shapes of all inputs must match: values[0].shape = [180,1,120,2000,1] != values[1].shape = [1,1,1,1,1] [Op:Pack] name: packed